In [157]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

gender_submission.csv
test.csv
train.csv



In [158]:
df_test = pd.read_csv("../input/test.csv")
df_train = pd.read_csv("../input/train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [159]:
#Converts Embarked location to numbers to enable correlation calcluation
df_test["Embarked"] = df_test["Embarked"].replace("C",1)
df_test["Embarked"] = df_test["Embarked"].replace("Q",2)
df_test["Embarked"] = df_test["Embarked"].replace("S",3)

df_train["Embarked"] = df_train["Embarked"].replace("C",1)
df_train["Embarked"] = df_train["Embarked"].replace("Q",2)
df_train["Embarked"] = df_train["Embarked"].replace("S",3)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,3.0


In [160]:
df_test["Sex"] = df_test["Sex"].replace("male",1)
df_test["Sex"] = df_test["Sex"].replace("female",0)

df_train["Sex"] = df_train["Sex"].replace("male",1)
df_train["Sex"] = df_train["Sex"].replace("female",0)
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,3.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,3.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,3.0


In [161]:
df_train_num = df_train.iloc[:,0:3]
df_train_num["Sex"] = df_train["Sex"] 
df_train_num["SibSp"] = df_train["SibSp"] 
df_train_num["Parch"] = df_train["Parch"] 
df_train_num["Fare"] = df_train["Fare"] 
df_train_num["Embarked"] = df_train["Embarked"] 
df_train_num.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,1,0,3,1,1,0,7.2500,3.0
1,2,1,1,0,1,0,71.2833,1.0
2,3,1,3,0,0,0,7.9250,3.0
3,4,1,1,0,1,0,53.1000,3.0
4,5,0,3,1,0,0,8.0500,3.0


In [162]:
df_tests = df_test.iloc[:,0:2]
df_tests["Pclass"] = df_test["Pclass"]
df_tests["Sex"] = df_test["Sex"]
df_tests["SibSp"] = df_test["SibSp"]
df_tests["Parch"] = df_test["Parch"]
df_tests["Fare"] = df_test["Fare"]
df_tests["Embarked"] = df_test["Embarked"]

df_tests.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,892,3,1,0,0,7.8292,2
1,893,3,0,1,0,7.0000,3
2,894,2,1,0,0,9.6875,2
3,895,3,1,0,0,8.6625,3
4,896,3,0,1,1,12.2875,3


In [163]:
X = df_train_num.drop(["Survived"], axis=1)
Y = df_train["Survived"]

In [164]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
X = my_imputer.fit_transform(X)

In [165]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [172]:
my_imputer_test = SimpleImputer()
df_tests = my_imputer.fit_transform(df_tests)
df_submission = df_test.iloc[:,:]

In [173]:
df_submission["Survived"] = clf.predict(df_tests)

In [175]:
df_submission = df_submission.drop(["Pclass"], axis=1)
df_submission = df_submission.drop(["Name"], axis=1)
df_submission = df_submission.drop(["Sex"], axis=1)
df_submission = df_submission.drop(["Age"], axis=1)
df_submission = df_submission.drop(["SibSp"], axis=1)
df_submission = df_submission.drop(["Parch"], axis=1)
df_submission = df_submission.drop(["Ticket"], axis=1)
df_submission = df_submission.drop(["Fare"], axis=1)
df_submission = df_submission.drop(["Cabin"], axis=1)
df_submission = df_submission.drop(["Embarked"], axis=1)
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [180]:
df_submission.to_csv("AS_Submission", index=False)